<a href="https://colab.research.google.com/github/telsayed/IR-in-Arabic/blob/master/Summer2021/labs/day1/IR_in_Arabic_Lab1_BooleanRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **IR in Arabic** - Summer 2021 lab day1


This is one of a series of Colab notebooks created for the **IR in Arabic** course. It demonstrates how can we build a Term-Document incidence matrix and a boolean retrieval model.

The **learning outcomes** of the this notebook are:


*   What is Colab and how can we run our code?
*   Build a Term-Document incidence Matrix and visualize it.
*   Build a boolean retrieval model that can answer boolean queries.



# What is colab?

**[Colaboratory](https://research.google.com/colaboratory/faq.html)**, or “Colab” for short, is a product from Google Research. Colab allows anybody to write and execute arbitrary python code through the browser, and is especially well suited to machine learning, data analysis and education. More technically, Colab is a hosted Jupyter notebook service that requires no setup to use, while providing free access to computing resources including GPUs.

# How can you have your own copy of this lab?

You have to make a copy of this notebook by clicking **open in colab**, then go to **File->Save a copy in Drive**. This will create your version of this lab saved in your Google Drive.

### **Running a simple code on colab.**

In [ ]:
print("مرحبا بكم في دورة استرجاع المعلومات")

### **Build a Term-Document incidence Matrix and visualize it.**

Let's create a set of textual documents to create our collection.

In [ ]:
 d0 = "هذا هو اليوم الأول من دورة استرجاع المعلومات"
 d1 = "الدورة باللغة العربية للطلاب العرب"
 d2 = "اليوم هو 30 مايو 2021"
 d3 = "نأمل أن تفيد هذه الدورة الطلاب العرب"
 d4 = "هل أنتم سعداء بهذه التجربة"
 collection = {"doc0": d0,
         "doc1": d1, 
         "doc2": d2, 
         "doc3": d3,
         "doc4": d4}
collection

{'doc0': 'هذا هو اليوم الأول من دورة استرجاع المعلومات',
 'doc1': 'الدورة باللغة العربية للطلاب العرب',
 'doc2': 'اليوم هو 30 مايو 2021',
 'doc3': 'نأمل أن تفيد هذه الدورة الطلاب العرب',
 'doc4': 'هل أنتم سعداء بهذه التجربة'}

Let's define our boolean operators.

In [ ]:
boolean_operators = {'AND', 'OR', 'NOT'}

We need to implement some helper functions below

In [ ]:
  #list of terms
  def get_terms(data):
        terms=[]
        for doc in data:
            for term in data[doc].split() :
                terms.append(term)
        return terms

   #list of unique terms
  def get_unique_terms(terms):
        unique_terms=[]
        for d in terms :
            if d not in unique_terms:
               unique_terms.append(d)
        return unique_terms

  #document collection terms
  def get_document_collection_terms(data):
        docs_colllection={}
        for doc in data:
            if doc not in boolean_operators :
                docs_colllection[doc]=get_unique_terms(data[doc].split())
        return docs_colllection
           
  def display_dict(dic):
        print("\n")
        for i in dic:
            print (i , " : " ,dic[i])
        print("\n")

Let's check the output of our helper functions.

In [ ]:
#print our collection
print("### documents content ###")
display_dict(collection)

In [ ]:
#print the terms available in the collection
print ("\n### Terms in 5 docs ###\n" , *get_terms(collection) ,sep= " | ")

In [ ]:
#print the unique_terms available in the collection
terms=get_terms(collection)
print ("\n### Unique Terms in 5 DOCs###\n", *get_unique_terms(terms) ,sep=" | ")

In [ ]:
#print the unique terms in each collection in a dictionary format
print ("\n###Document terms Collection ###" )
display_dict(get_document_collection_terms(collection))

Next, we will implement a function to build a term-document incidence matrix

In [ ]:
#this function takes the collection of documents in a form of dictionary as an input
def term_document_incidence_matrix(collection):
        ## list of terms from the data file collection
        terms=get_terms(collection)
        #list of unique terms
        unique_terms=get_unique_terms(terms)
        #Document collection terms
        docs_terms=get_document_collection_terms(collection)
        #TermDocumentIncidenceMatrix 
        term_docs_matrix={}
        for term in unique_terms:
            vector=[]
            for c in docs_terms:

                if term in docs_terms[c]:
                    vector.append(1)
                else :
                    vector.append(0)

            term_docs_matrix[term]=vector
        return term_docs_matrix

#this fucntion takes a term and a terms-document incidence matrix and returns the incidence vector of the term 
#this function just for explanation and display purposes   
def term_incidence_vector(term,term_docs_incid_matrix):
    try:
     return term_docs_incid_matrix[term]
    except:
     return "term not found"

Let's build the term-document incidence matrix

In [ ]:
#build the term-document incidence matrix
term_docs_incid_matrix=term_document_incidence_matrix(collection)
print("Term-Document incidence Matrix\n")
#formatted Display 
display_dict(term_docs_incid_matrix)


Let's check the term incidence vector for the word العرب

In [ ]:
print("Incidence Vector of 'العرب' ",term_incidence_vector('العرب',term_docs_incid_matrix))

### **Buidling a Boolean retrieval model**

First, we need to implement our function that splits the query to differentiate between the terms and the boolean operators, and considers only boolean operators defined previously and the terms available in our collection.


In [ ]:
#Query Filteration
#input : Query 
#output : List of terms of a given query which match with the terms in our collection and the our defined boolean operators
def query_filteration(query,collection):
    terms= get_terms(collection)
    unique_terms=get_unique_terms(terms)
    qterms=[]
    splitted_query=query.split()
    for qterm in splitted_query:
        if qterm in unique_terms or qterm in boolean_operators:
          qterms.append(qterm)
    return qterms

Let's test our query filteration function. 

In [ ]:
query="استرجاع AND المعلومات AND دورة"
qterms=query_filteration(query,collection)
print(qterms)

If a term is not defined in our collection,or a boolean operator is not defined in our set of our boolean operators it will not be considered.

In [ ]:
#and should be capitalized and اليمن is not in our collection
query="استرجاع and المعلومات AND اليمن"
qterms=query_filteration(query,collection)
print(qterms)

In [ ]:
# Boolean Operator Processing 
# input : Boolean Operator ,Next term Incedence Vector ,Previous term Incedence Vector 
def boolean_operator_processing(bool_operator,prevV,nextV):
    if bool_operator == "AND":
      return [a & b for a, b in zip(prevV, nextV)]
    elif bool_operator=="OR" :
      return [a | b for a, b in zip(prevV, nextV)]
    elif bool_operator == "NOT":
      return [1-a for a in prevV]

Let's test our BooleanOperatorProcessing function

In [ ]:
v1=term_incidence_vector('استرجاع',term_docs_incid_matrix)
v2=term_incidence_vector('الدورة',term_docs_incid_matrix)
v3=[]
print('v1',v1)
print('v2',v2)

In [ ]:
print(boolean_operator_processing("AND",v1,v2))

In [ ]:
print(boolean_operator_processing("OR",v1,v2))

In [ ]:
print(boolean_operator_processing("NOT",v1,v3))

In [ ]:
# Boolean retrieval function
# input : Query 
def boolean_retrieval(query,collection):
        #build a terms_documents incidence matrix
        term_docs_incid_matrix=term_document_incidence_matrix(collection)
        bitwiseop=""
        #get the query terms
        qterms=query_filteration(query,collection)
        result=[]
        result_set={}
        has_previous_term=False
        has_not_operation=False
        inc_vec_prev=[]
        inc_vec_next=[]
        for term in qterms :
            if term not in boolean_operators:
                if  has_not_operation:
                    if has_previous_term:
                         inc_vec_next=boolean_operator_processing("NOT",term_docs_incid_matrix[term],inc_vec_next)
                    else :
                        inc_vec_prev=boolean_operator_processing("NOT",term_docs_incid_matrix[term],inc_vec_next)
                        result=inc_vec_prev
                    has_not_operation=False
                    
                elif  has_previous_term:
                    
                    inc_vec_next=term_docs_incid_matrix[term]
                else :
                    
                    inc_vec_prev=term_docs_incid_matrix[term]
                    result= inc_vec_prev
                    has_previous_term=True    
            
            elif term =="NOT":
                 has_not_operation=True
            
            else :
                bitwiseop=term
                
            if  len(inc_vec_next)!= 0  :
                result = boolean_operator_processing(bitwiseop,inc_vec_prev,inc_vec_next)
                inc_vec_prev=result
                has_previous_term=True
                inc_vec_next= []
    
        for i,doc in zip(result,collection):
            result_set[doc]=i
        return result_set

Let's use our boolean retrieval model 

In [ ]:
#print the collection to check whether the boolean retrieval output is correct or not easily
collection

In [ ]:
query1 = "استرجاع OR الدورة OR سعداء"
print("query1 boolean retrieval ",boolean_retrieval(query1,collection))

In [ ]:
v1=term_incidence_vector("استرجاع",term_docs_incid_matrix)
v1

In [ ]:
v2=term_incidence_vector("الدورة",term_docs_incid_matrix)
v2

In [ ]:
Qpart1=boolean_operator_processing("OR",v1,v2)
Qpart1

In [ ]:
v3=term_incidence_vector("سعداء",term_docs_incid_matrix)
v3

In [ ]:
boolean_operator_processing("OR",Qpart1,v3)

In [ ]:
#print the collection to check whether the boolean retrieval output is correct or not easily
collection

In [ ]:
query2 = "استرجاع AND المعلومات AND NOT العربية"
print("query2 boolean retrieval ",boolean_retrieval(query2,collection))

In [ ]:
query3 = "العرب AND NOT المعلومات"
print("query3 boolean retrieval result ",boolean_retrieval(query3,collection))

### **Exercise1**

Find the documents that contains the word **سعداء** or **الدورة** but does not contain 
the word **العرب**

### **Exercise2**
Using the sentences given below create your documents, build your term-document incidence matrix, and use the given three queries to retrieve their relevant documents.


1. قضينا في الخليل ستة أشهر ثم بدأ أهل البلد يتوزعون منهم من أراد اللحاق بأقرباء له في طولكرم أو نابلس أو جنين و منهم من تسلل عائدا الى الجليل و منهم من ذهب الى سوريا
2.   الحرب تخيف كثيراً  لكنها تعلم تعلمك الحرب أن الكلمات قد لا تعني مدلولاتها بشكل كامل 
3. نعم زرت بيروت زرتها بعد خمس سنين من الغياب
4.ويلجم الجميع لا أحد يستطيع تفسير ما حصل يتمتمون الجيوش العربية ستتدخل أجل ستتدخل بالتأكيد هل يترك العرب فلسطين تضيع
5. ترحل إلى بيروت تظنها نهاية الحرب وبداية الاستقرار يخونها حدسها الحرب هنا غير تلك في فلسطين لكنها الحرب التي لا تعبأ بك وبمن تكون
**ملحوظة:** الجمل مقتبسة من رواية الطنطورية لرضوى عاشور


*   **query1:** بيروت OR فلسطين OR سوريا
*   **query2:** الحرب AND فلسطين
*   **query3:** الحرب AND NOT فلسطين  



In [ ]:
#create your collection

In [ ]:
#build the term incidence matrix

In [ ]:
#boolean retrieval query1


In [ ]:
#boolean retrieval query2


In [ ]:
#boolean retrieval query3


### **References**


*   [Information retrieval models.](https://github.com/YoucefBYu/Information-Retrieval-Models)

